# Greedy Algorithm

In [2]:
from typing import List

## 605. Can Place Flowers
Given an integer array flowerbed containing 0's and 1's, where 0 means empty and 1 means not empty, and an integer n, return if n new flowers can be planted in the flowerbed without violating the no-adjacent-flowers rule.

A single scan is enough.  
The tricky part is to set the conditionals, and the corner case n == 0.  
An improvement is to make early termination when n decreases to 0.

In [3]:
class Solution:
    def canPlaceFlowers(self, flowerbed: List[int], n: int) -> bool:
        if n == 0:  # if n is 0, just return true
            return True
        i = 0
        while i < len(flowerbed):
            if flowerbed[i] == 0:
                if i + 1 >= len(flowerbed) or flowerbed[i+1] == 0:
                    n = n - 1
                    # check for early termination:
                    if n == 0:
                        return True 
                    i = i + 2
                else:
                    i = i + 3
            else:
                i = i + 2
            
        return False

## 452. Minimum Number of Arrows to Burst Balloons
Given an array points where points[i] = [xstart, xend], return the minimum number of arrows that must be shot to burst all balloons.

A sort (on a key) and a scan is enough to solve it.  
Can use the x_start to sort, but using x_end to sort writes more conveniently.  
*sort()* and *sorted()* in Python both take the *key* argument, pass in a lambda. Also, can directly use *min()*.

In [4]:
# sort on x_start: have to do a min() comparison
class Solution:
    def findMinArrowShots(self, points: List[List[int]]) -> int:
        if len(points) == 0:
            return 0
        points.sort(key=lambda x: x[0])  # sort in terms of x_start
        count = 1
        x_end = points[0][1]
        for i in range(1, len(points)):
            if points[i][0] <= x_end:
                x_end = min(x_end, points[i][1])  # compare the end here!
            else:
                count = count + 1
                x_end = points[i][1]
        return count

In [5]:
# sort on x_end: don't need to compare x_end any more, so a bit faster
class Solution:
    def findMinArrowShots(self, points: List[List[int]]) -> int:
        if not points:  # also includes case of size==0
            return 0
        points.sort(key=lambda x: x[1])  # sort in terms of x_start
        count = 1
        curr_first_end = points[0][1]
        for x_start, x_end in points:
            if x_start > curr_first_end:
                count += 1
                curr_first_end = x_end
        return count

## 763. Partition Labels
A string S of lowercase English letters is given. We want to partition this string into as many parts as possible so that each letter appears in at most one part, and return a list of integers representing the size of these parts.

First scan and get some information, which is the intervals of the characters (or the index of its last occurrence).  
Then scan the intervals/string to get the overlapping intervals and compute the length.  
**enumerate()** is a great function to use, it might speed up the program a bit.

In [7]:
# my solution: get both start and end indices of a char
class Solution:
    def partitionLabels(self, S: str) -> List[int]:
        # pre-process: a map from char to 2-size list, x_start and x_end
        map = dict()
        for i, c in enumerate(S):  # enumerate: iterable with counters
            if not c in map:
                map[c] = [i, i]
            else:
                map[c][1] = i
        ranges = list(map.values())  # an easy way to get values as list
        # sort based on x_start
        ranges.sort(key = lambda x: x[0])
        # if two intervals are a bit overlapped, they should be 1 part
        partitions = []
        curr_last_end = ranges[0][1]  # the curr end of partition
        curr_start = 0  # the start of curr partition
        for start, end in ranges:
            if start > curr_last_end:
                partitions.append(curr_last_end - curr_start + 1)
                curr_start = curr_last_end + 1
                curr_last_end = end
            else:
                curr_last_end = max(curr_last_end, end)
        partitions.append(curr_last_end - curr_start + 1)
        return partitions

In [8]:
# leetcode solution: very concise
class Solution:
    def partitionLabels(self, S: str) -> List[int]:
        last = {c: i for i, c in enumerate(S)}  # get the last occurrence indices in 1 line
        start = end = 0
        partitions = []
        for i, c in enumerate(S):
            end = max(end, last[c])
            if i == end:  # at the end of the curr partition
                partitions.append(end - start + 1)
                start = i + 1
        return partitions

## 122. Best Time to Buy and Sell Stock II
Say you have an array prices for which the ith element is the price of a given stock on day i.
Design an algorithm to find the maximum profit. You may complete as many transactions as you like.

A single scan would do. But writing the conditionals could be tricky.

In [9]:
# my solution:
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        profit = 0
        buy = -1  # if there is a buy
        for i, price in enumerate(prices):
            if buy < 0 and i+1 < len(prices) and prices[i+1] > price:  # no buy and should buy
                buy = price
            elif buy >= 0 and (i+1 >= len(prices) or prices[i+1] < price):  # already bought one and should sell
                profit += price - buy
                buy = -1
        return profit

In [12]:
# leetcode best solution: as long as there's an increase, add it to the profit!
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        profit = 0
        for i in range(1, len(prices)):
            if prices[i] > prices[i-1]:
                profit += prices[i] - prices[i-1]
        return profit

## 406. Queue Reconstruction by Height
You are given an array of people, people, which are the attributes of some people in a queue (not necessarily in order). Each people[i] = [hi, ki] represents the ith person of height hi with exactly ki other people in front who have a height greater than or equal to hi.  
Reconstruct and return the queue that is represented by the input array people. The returned queue should be formatted as an array queue, where queue[j] = [hj, kj] is the attributes of the jth person in the queue (queue[0] is the person at the front of the queue).



The following strategy could be continued recursively:  
1. Sort the tallest guys in the ascending order by k-values and then insert them one by one into output queue at the indexes equal to their k-values.
2. Take the next height in the descending order. Sort the guys of that height in the ascending order by k-values and then insert them one by one into output queue at the indexes equal to their k-values.
3. And so on and so forth.

**sort(key = lambda x: (x[0], x[1])**: by assigning a tuple to the lambda, it means to sort first on the 1st element of the tuple, if that one is equal, sort based on the second element then.  
**lst.insert(index, element)**: a way to insert into a place in the list, takes O(k), where k is the current length.

In [13]:
# leetcode solution
class Solution:
    def reconstructQueue(self, people: List[List[int]]) -> List[List[int]]:
        people.sort(key = lambda x: (-x[0], x[1]))
        output = []
        for p in people:
            output.insert(p[1], p)
        return output

## 665. Non-decreasing Array
Given an array nums with n integers, your task is to check if it could become non-decreasing by modifying at most 1 element.

Don't know why this one is an "easy" problem, it's very tricky, and prone to error.  
It requires to consider 4 nodes at a time to see if a decrease situation can be fixed.

**Algorithm**  
As before, let p be the unique problem index for which $A[p]>A[p+1]$. If this is not unique or doesn't exist, the answer is False or True respectively. We analyze the following cases:  
If $p = 0$, then we could make the array good by setting $A[p] = A[p+1]$.  
If $p = len(A) - 2$, then we could make the array good by setting $A[p+1] = A[p]$.  
Otherwise, $A[p-1], A[p], A[p+1], A[p+2]$ all exist, and:  
We could change $A[p]$ to be between $A[p-1]$ and $A[p+1]$ if possible, or;  
We could change $A[p+1]$ to be between $A[p]$ and $A[p+2]$ if possible.

In [ ]:
# my solution:
class Solution:
    def checkPossibility(self, nums: List[int]) -> bool:
        if len(nums) == 1:
            return True
        hp = 1  # one chance
        for i in range(len(nums) - 1):
            if nums[i] > nums[i+1]:
                hp -= 1
                if hp < 0:
                    return False
                if i > 0 and i+1 < len(nums) - 1: # not first (if i is first, ignore), i+1 not last
                    # now there are i-1, i, i+1, i+2:
                    if nums[i+2] < nums[i+1]:
                        return False
                    elif nums[i+1] < nums[i-1]:  # otherwise changing nums[i] would do
                        if nums[i+2] < nums[i]:
                            return False
        return True

In [14]:
# leetcode
class Solution(object):
    def checkPossibility(self, A):
        p = None
        for i in xrange(len(A) - 1):
            if A[i] > A[i+1]:
                if p is not None:
                    return False
                p = i

        return (p is None or p == 0 or p == len(A)-2 or
                A[p-1] <= A[p+1] or A[p] <= A[p+2])